<a href="https://colab.research.google.com/github/Meyyaou/pythondaily/blob/main/STEP4-Webscrape/webscrape_project_(minione).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#import all tools
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml
import urllib.parse

In [58]:
#from search page to individually each page
header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7,ar;q=0.6'}

In [22]:
#to get readable html post parsing
#soup.prettify()

In [59]:
def get_book_info(url):
    page=requests.get(url,headers=header)
    soup=BeautifulSoup(page.text, 'lxml')
    #get title
    title_elem= soup.select_one("#productTitle")
    title=title_elem.text if title_elem else None
    #print(title)
    #get author
    author_elem=soup.select_one('span.author')
    author=author_elem.text if author_elem else None
    #print(author)
    #get date de sortie
    date_elem=soup.select_one("#rpi-attribute-book_details-publication_date > div.a-section.a-spacing-none.a-text-center.rpi-attribute-value > span")

    date=date_elem.text.strip() if date_elem else None
    #print(date)
    #get price prime deal
    full_price_elem= soup.select_one('span.a-price-whole')
    frac_price_elem=soup.select_one('span.a-price-fraction')
    curr_elem=soup.select_one("span.a-price-symbol")
    price=full_price_elem.text+frac_price_elem.text+curr_elem.text if full_price_elem and frac_price_elem and curr_elem else None
    #print(price)

    #get ordre in top vente
    order_bs_elem= soup.select_one("div.zg-badge-wrapper > a > i")
    order_bs=order_bs_elem.text if order_bs_elem else None

    cat_bs_elem=soup.select_one("div.zg-badge-wrapper > a > span > span")
    cat_bs=cat_bs_elem.text if cat_bs_elem else None
    #print(order_bs, cat_bs)

    #get price livraison ?wherfe is itttt
    #get price kindle?
    return {
        'title': title,
        'author': author,
        'date': date,
        'price': price,
        'order_bs': order_bs,
        'cat_bs': cat_bs
    }

In [60]:

counter = 0

def search_res_listing(url_search, books_data):
    global counter
    response = requests.get(url_search, headers=header)
    search_soup = BeautifulSoup(response.text, 'lxml')
    url_elem_list = search_soup.select('[data-asin] h2 a')

    for url_elem in url_elem_list:
        if counter >= 20:
            return books_data
        url = urllib.parse.urljoin(url_search, url_elem.attrs.get("href"))
        book_info = get_book_info(url)
        books_data.append(book_info)
        counter += 1

    next_book_el = search_soup.select_one('a:contains("Next")')
    if next_book_el and counter < 20:
        next_book_url = next_book_el.attrs.get('href')
        next_book_url = urllib.parse.urljoin(url_search, next_book_url)
        books_data = search_res_listing(next_book_url, books_data)

    return books_data

In [61]:
s_url="https://www.amazon.com/s?k=manga&rh=n%3A4367&ref=nb_sb_noss"
books_data = search_res_listing(s_url, [])
dataframe = pd.DataFrame(books_data)
dataframe.to_csv("mangas_amazon_test1.csv", index=False, sep=',')

In [ ]:
#tronquer what we dont need! => was when we searched for infos from results page, not consequent!
#for header in soup.find_all('header'):
#  header.decompose()
#soup.find('div', class_='navLeftFooter').decompose()
#soup.find_all('div', class_='navLeftFooter')
#soup.find('div', class_='rhf-frame').decompose()
#soup.find('div', class_='copilot-secure-display').decompose()

[]

In [ ]:
#get the title
#titles=[]
#pattern =re.compile(r'\d+')

#for title in soup.find_all('span', class_='a-text-normal'):
#  if not pattern.match(title.string):
#    titles.append(title.string)
#    title
#titles

[]

In [ ]:
#get auteur
#auteurs=[]

#for auteur in soup.find_all('span', class_='a-size-base'):
#  auteurs.append(auteur.string)
#auteurs


[]

In [62]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     20 non-null     object
 1   author    20 non-null     object
 2   date      20 non-null     object
 3   price     20 non-null     object
 4   order_bs  6 non-null      object
 5   cat_bs    6 non-null      object
dtypes: object(6)
memory usage: 1.1+ KB


In [63]:
dataframe["author"]

0                   \nYoshitoki Oima \n(Author) \n
1                 \nTatsuki Fujimoto \n(Author) \n
2                 \nKoyoharu Gotouge \n(Author) \n
3     \nKentaro Miura \n(Author, Illustrator),  \n
4                   \nTsugumi Ohba \n(Author),  \n
5                   \nTsugumi Ohba \n(Author),  \n
6                      \nKei Urana \n(Author),  \n
7               \nHye Young Im \n(Translator),  \n
8                  \nTakeru Hokazono \n(Author) \n
9                  \nNaoya Matsumoto \n(Author) \n
10                    \nTatsuya Endo \n(Author) \n
11                \nTatsuki Fujimoto \n(Author) \n
12                         \nJinushi \n(Author) \n
13                \nTatsuki Fujimoto \n(Author) \n
14                 \nShirow Masamune \n(Author) \n
15                          \nFujita \n(Author) \n
16                   \nKaiu Shirai \n(Author),  \n
17                   \nMokumokuren \n(Author),  \n
18                 \nShuichi Shigeno \n(Author) \n
19               \nKanehito Yam

In [66]:
df=dataframe

In [74]:
#DATA preprocessing: format des champs
#date
df['date']= pd.to_datetime(df['date'])
#
auths= df['author']
for i,auth in enumerate(auths):
  auth_name=""
  auth=auth.split(" ")
  auth_name+=auth[0]
  auth_name+=' '
  auth_name+=auth[1] if 'Author' not in auth[1] else ""
  auth_name=auth_name.split("\n")[1]
  df.loc[i,"author"]=auth_name
#print(df['author'])

IndexError: list index out of range

In [84]:
import numpy as np

In [105]:
df.head()
#cat_bs drop the in keep list of cats and if none=> No bs
df['cat_bs']=df['cat_bs'].fillna("No bs")
cats=df['cat_bs']
cat_bs = []
for cat in cats:
  if cat != 'No bs':
    words=str(cat).split(' ')
    filtr_words=[]
    i=0
    if words[0]=='in':
      i=1
    while i <len(words):
      if words[i]=="&":
        if i>0 and filtr_words:
          filtr_words[-1]=filtr_words[-1]+' '+words[i+1]
        i+=2
      else:
        filtr_words.append(words[i])
        i+=1
    cat_bs.append(filtr_words)
  else:
    cat_bs.append(["No bs"])
#print(cat_bs)
df['cat_bs']=cat_bs

In [106]:
df.head()

,title,author,date,price,order_bs,cat_bs
0,A Silent Voice Complete Series Box Set,Yoshitoki Oima,2017-12-19,32.26$,#1 Best Seller,"[Romance, Manga]"
1,Chainsaw Man Box Set: Includes volumes 1-11,Tatsuki Fujimoto,2023-09-26,55.99$,#1 Best Seller,"[Teen Young, Adult, Manga]"
2,Demon Slayer Complete Box Set: Includes volu...,Koyoharu Gotouge,2021-11-09,115.52$,#1 Best Seller,"[Science, Fiction, Manga]"
3,Berserk Deluxe Volume 1,Kentaro Miura,2019-03-26,28.48$,#1 Best Seller,"[Dark, Horse, Comics Graphic, Novels]"
4,Death Note Complete Box Set: Volumes 1-13 wi...,Tsugumi Ohba,2008-10-07,59.43$,None,[No bs]


In [ ]:
#order_bs if none=> No bs
df['order_bs']=df['order_bs'].fillna("No bs")

In [ ]:
#DATA ANALYSIS some basic plots
#annee de publication
df['date'].hist()

#price
import matplotlib.pyplot as plt
plt.figure()
plt.plot(df['price', c='green')
plt.show()

In [ ]:
#variation analysis

In [13]:
#email alert stuff
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [14]:
def email_alert(subject,body,to):
  msg=MIMEMultipart()
  msg.attach(MIMEText(body, 'plain'))
  msg['subject']=subject
  msg['to']=to
  user='x'
  msg['from']=user

  pwd='x'

  server=smtplib.SMTP("smtp.gmail.com",587)
  server.starttls()
  server.login(user,pwd)
  server.send_message(msg)
  server.quit()

In [15]:
#email_alert('hey', "hello world it's a test man", 'rec@gmail.com')